<a href="https://colab.research.google.com/github/cdss8/AI-ML/blob/main/JM_Laboratory2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Juan Manuel Sialo
#Laboratory 2

In [49]:
#PySpark enables to perform real-time, large-scale data processing.
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [101]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint

from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.tree import RandomForest, RandomForestModel

from pyspark.mllib.util import MLUtils

from pyspark import SparkContext

from time import time

In [63]:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [64]:
data = pd.read_table('/content/gdrive/My Drive/AI-ML/covtype.data')
data.head()

,"2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5"
0,"2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,..."
1,"2804,139,9,268,65,3180,234,238,135,6121,1,0,0,..."
2,"2785,155,18,242,118,3090,238,238,122,6211,1,0,..."
3,"2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,..."
4,"2579,132,6,300,-15,67,230,237,140,6031,1,0,0,0..."


In [65]:
#Forest CoverTypes classes (Roosevelt National Forest of northern Colorado)
CoverTypes={1.0: 'Spruce/Fir',
            2.0: 'Lodgepole Pine',
            3.0: 'Ponderosa Pine',
            4.0: 'Cottonwood/Willow',
            5.0: 'Aspen',
            6.0: 'Douglas-fir',
            7.0: 'Krummholz' }
print('Tree CoverTypes in the Forest:')
CoverTypes

Tree CoverTypes in the Forest:


{1.0: 'Spruce/Fir',
 2.0: 'Lodgepole Pine',
 3.0: 'Ponderosa Pine',
 4.0: 'Cottonwood/Willow',
 5.0: 'Aspen',
 6.0: 'Douglas-fir',
 7.0: 'Krummholz'}

In [66]:
#Attribute information 
col_labels="""
Elevation, Aspect, Slope, Horizontal_Distance_To_Hydrology,
Vertical_Distance_To_Hydrology, Horizontal_Distance_To_Roadways,
Hillshade_9am, Hillshade_Noon, Hillshade_3pm,
Horizontal_Distance_To_Fire_Points, Wilderness_Area (4 binarycolumns), 
Soil_Type (40 binary columns), Cover_Type
"""

In [67]:
# Breaking-up attributes that are made out of several binary features.
col=[a.strip() for a in col_labels.split(',')]
colDict={a:[a] for a in col}
colDict['Soil_Type (40 binary columns)'] = ['ST_'+str(i) for i in range(40)]
colDict['Wilderness_Area (4 binarycolumns)'] = ['WA_'+str(i) for i in range(4)]
Columns=[]
for item in col:
    Columns=Columns+colDict[item]
print(Columns)

['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'WA_0', 'WA_1', 'WA_2', 'WA_3', 'ST_0', 'ST_1', 'ST_2', 'ST_3', 'ST_4', 'ST_5', 'ST_6', 'ST_7', 'ST_8', 'ST_9', 'ST_10', 'ST_11', 'ST_12', 'ST_13', 'ST_14', 'ST_15', 'ST_16', 'ST_17', 'ST_18', 'ST_19', 'ST_20', 'ST_21', 'ST_22', 'ST_23', 'ST_24', 'ST_25', 'ST_26', 'ST_27', 'ST_28', 'ST_29', 'ST_30', 'ST_31', 'ST_32', 'ST_33', 'ST_34', 'ST_35', 'ST_36', 'ST_37', 'ST_38', 'ST_39', 'Cover_Type']


# **class pyspark.SparkContext**

***datatype (master=None, appName=None, sparkHome=None, pyFiles=None, environment=None, batchSize=0, serializer=PickleSerializer(), conf=None, gateway=None, jsc=None, profiler_cls=<class 'pyspark.profiler.BasicProfiler'>)***

 A SparkContext ***represents the connection to a Spark cluster, and can be used to create RDD*** and broadcast variables on that cluster.


In [92]:
#getting the local spark created, or creatingone if the SparkContext was created already

sc = SparkContext.getOrCreate() 

In [93]:
#reading the file into RRD

inputRDD=sc.textFile('/content/gdrive/My Drive/AI-ML/covtype.data')
inputRDD.first()

'2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5'

In [94]:
#transforming the text RDD into an RDD of LabeledPoints
newData=inputRDD.map(lambda line: [float(x.strip()) for x in line.split(',')])\
     .map(lambda V:LabeledPoint(V[-1],V[:-1]))
newData.first()

LabeledPoint(5.0, [2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])

In [95]:
#counting the number of examples of each type
total=newData.cache().count()
print(('data_size: numbers of trees in the area:=',total))
counts=newData.map(lambda lp:(lp.label,1)).reduceByKey(lambda x,y:x+y).collect()

counts.sort(key=lambda x:x[1],reverse=True)
print('              tree  (type):   percent (%)')
print('---------------------------------------------------------')
print(('\n'.join(['%20s (%3.1f):\t%4.2f'%(CoverTypes[a[0]],a[0],100.0*a[1]/float(total)) for a in counts])))

('data_size: numbers of trees in the area:=', 581012)
              tree  (type):   percent (%)
---------------------------------------------------------
      Lodgepole Pine (2.0):	48.76
          Spruce/Fir (1.0):	36.46
      Ponderosa Pine (3.0):	6.15
           Krummholz (7.0):	3.53
         Douglas-fir (6.0):	2.99
               Aspen (5.0):	1.63
   Cottonwood/Willow (4.0):	0.47


# **Making the problem binary**
The implementation of BoostedGradientTrees in MLLib supports only binary problems. Our CovType data has 7 classes. To make the problem binary we will choose one of the trees.  Lodgepole Pine: type=2.0. We therefor transform the dataset to a new dataset where the type is 1.0 is the class is Lodgepole Pine and is 0.0 otherwise.

In [97]:
Type=2.0
newData=inputRDD.map(lambda line: [float(x) for x in line.split(',')])\
    .map(lambda V:LabeledPoint(1.0*(V[-1]==Type),V[:-1]))

In [99]:
#reducing the data top se better the effect of the study

visualitationData=newData.sample(False,1).cache()
(trainingData,testData)=visualitationData.randomSplit([0.7,0.3])

print('Sizes: visualitationData=%d, trainingData=%d, testData=%d'%(visualitationData.count(),trainingData.cache().count(),testData.cache().count()))

Sizes: visualitationData=581012, trainingData=406835, testData=174177


In [100]:
counts=testData.map(lambda lp:(lp.label,1)).reduceByKey(lambda x,y:x+y).collect()
counts.sort(key=lambda x:x[1],reverse=True)
counts

[(0.0, 89176), (1.0, 85001)]

In [102]:
errors={}

for depth in [1,3,6,10,15]:

    start=time()
    model = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=100, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=depth, maxBins=32)
    
    #print model.toDebugString()
    errors[depth]={}
    dataSets={'train':trainingData,'test':testData}
    for name in list(dataSets.keys()):  # Calculate errors on train and test sets
        data=dataSets[name]
        Predicted=model.predict(data.map(lambda x: x.features))
        LabelsAndPredictions=data.map(lambda lp: lp.label).zip(Predicted)
        Err = LabelsAndPredictions.filter(lambda v_p:v_p[0] != v_p[1]).count()/float(data.count())
        errors[depth][name]=Err
    print(depth,errors[depth],int(time()-start),'seconds')
print(errors)

1 {'train': 0.355234923248983, 'test': 0.35710799933401083} 91 seconds
3 {'train': 0.269200044243981, 'test': 0.2705236627109205} 102 seconds
6 {'train': 0.25252743741320194, 'test': 0.25340888865923744} 142 seconds
10 {'train': 0.2222842184177861, 'test': 0.22343363360259966} 228 seconds
15 {'train': 0.17999434660243097, 'test': 0.18335945618537464} 445 seconds
{1: {'train': 0.355234923248983, 'test': 0.35710799933401083}, 3: {'train': 0.269200044243981, 'test': 0.2705236627109205}, 6: {'train': 0.25252743741320194, 'test': 0.25340888865923744}, 10: {'train': 0.2222842184177861, 'test': 0.22343363360259966}, 15: {'train': 0.17999434660243097, 'test': 0.18335945618537464}}


In [103]:
RF10p_10={1: {'test': 0.41132990702777616, 'train': 0.4119253191646373}, 3: {'test': 0.27504764104637064, 'train': 0.2733131626202248}, 6: {'test': 0.2566264364497315, 'train': 0.2549627333776105}, 10: {'test': 0.23167985216838943, 'train': 0.2252724276191179}, 15: {'test': 0.20182479644280188, 'train': 0.18323370968932182}, 20: {'test': 0.19091066581971472, 'train': 0.15942242884903943}}
RF10p_100={1: {'test': 0.3960270254663048, 'train': 0.39411605539566574}, 3: {'test': 0.2815730207310735, 'train': 0.2798071483039382}, 6: {'test': 0.26396027025466307, 'train': 0.26189949081248615}, 10: {'test': 0.23081365132528728, 'train': 0.22281258455710526}, 15: {'test': 0.1997459144193567, 'train': 0.18001131527808525}, 20: {'test': 0.17803314661892938, 'train': 0.14658204806533343}}
RFall_10={1: {'test': 0.4347141897208231, 'train': 0.4350283735527522}, 3: {'test': 0.29120314267828323, 'train': 0.29215788917479535}, 6: {'test': 0.2565544254216944, 'train': 0.25831115305498004}, 10: {'test': 0.23009550939300133, 'train': 0.23181032851388447}, 15: {'test': 0.1900252126419288, 'train': 0.18882605500709032}, 20: {'test': 0.160103147847162, 'train': 0.154824487027302}}
RFall_100={1: {'test': 0.37008023248468, 'train': 0.37146866620954405}, 10: {'test': 0.21840810020732948, 'train': 0.2205764168958424}, 3: {'test': 0.289640992654449, 'train': 0.29019177031799515}, 6: {'test': 0.2532176270251954, 'train': 0.25488519094700574}, 15: {'test': 0.19286809595736248, 'train': 0.1926280373464277}}
def pp(R):
    return '\n'.join(['%s:%s'%(str(key),str(R[key])) for key in list(R.keys())])
print("RF10p_10=\n%s\nRF10p_100=\n%s\nRFall_10=\n%s\nRFall_100=\n%s\n"%(pp(RF10p_10),pp(RF10p_100),pp(RFall_10),pp(RFall_100)))

RF10p_10=
1:{'test': 0.41132990702777616, 'train': 0.4119253191646373}
3:{'test': 0.27504764104637064, 'train': 0.2733131626202248}
6:{'test': 0.2566264364497315, 'train': 0.2549627333776105}
10:{'test': 0.23167985216838943, 'train': 0.2252724276191179}
15:{'test': 0.20182479644280188, 'train': 0.18323370968932182}
20:{'test': 0.19091066581971472, 'train': 0.15942242884903943}
RF10p_100=
1:{'test': 0.3960270254663048, 'train': 0.39411605539566574}
3:{'test': 0.2815730207310735, 'train': 0.2798071483039382}
6:{'test': 0.26396027025466307, 'train': 0.26189949081248615}
10:{'test': 0.23081365132528728, 'train': 0.22281258455710526}
15:{'test': 0.1997459144193567, 'train': 0.18001131527808525}
20:{'test': 0.17803314661892938, 'train': 0.14658204806533343}
RFall_10=
1:{'test': 0.4347141897208231, 'train': 0.4350283735527522}
3:{'test': 0.29120314267828323, 'train': 0.29215788917479535}
6:{'test': 0.2565544254216944, 'train': 0.25831115305498004}
10:{'test': 0.23009550939300133, 'train': 0.2

In [104]:
Results={ID:globals()[ID] for ID in ['RF10p_10','RF10p_100','RFall_10','RFall_100']}
Results

{'RF10p_10': {1: {'test': 0.41132990702777616, 'train': 0.4119253191646373},
  3: {'test': 0.27504764104637064, 'train': 0.2733131626202248},
  6: {'test': 0.2566264364497315, 'train': 0.2549627333776105},
  10: {'test': 0.23167985216838943, 'train': 0.2252724276191179},
  15: {'test': 0.20182479644280188, 'train': 0.18323370968932182},
  20: {'test': 0.19091066581971472, 'train': 0.15942242884903943}},
 'RF10p_100': {1: {'test': 0.3960270254663048, 'train': 0.39411605539566574},
  3: {'test': 0.2815730207310735, 'train': 0.2798071483039382},
  6: {'test': 0.26396027025466307, 'train': 0.26189949081248615},
  10: {'test': 0.23081365132528728, 'train': 0.22281258455710526},
  15: {'test': 0.1997459144193567, 'train': 0.18001131527808525},
  20: {'test': 0.17803314661892938, 'train': 0.14658204806533343}},
 'RFall_10': {1: {'test': 0.4347141897208231, 'train': 0.4350283735527522},
  3: {'test': 0.29120314267828323, 'train': 0.29215788917479535},
  6: {'test': 0.2565544254216944, 'train':